In [76]:
import time
import pandas as pd
import os 
from selenium import webdriver
import time, random
import pickle

# Get audiobook titles

In [ ]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = \
    chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.audiobooks.com/browse')

In [ ]:
base_url = 'https://www.audiobooks.com/browse/listType/list/page/'
url_list = [base_url+str(i) for i in range(1,101)]

titles = []
for url in url_list:
    time.sleep(1 + 2*random.random())
    driver.get(url)
    title = [i.text for i in driver.find_elements_by_class_name('book-title')]
    titles.append(title)
    

In [ ]:
num_list = [i for i in range(0,20)]

book_titles = []
cnt = -1
for title in titles:
    cnt += 1
    for i in range(len(num_list)):
        sub_2 = i
        try:
            book = titles[cnt][sub_2]
            book_titles.append(book)
        except:
            pass

In [ ]:
driver.close()

In [ ]:
with open('book_titles_list.pickle', 'wb') as f:
    pickle.dump(book_titles, f)

In [77]:
with open('book_titles_list.pickle', 'rb') as f:
    titles = pickle.load(f)

In [78]:
len(titles)

2000

Titles that end with ... do not pull up the correct pages during scrape, need to clean them

In [79]:
for title in titles:
    if title.endswith("..."):
        print(title)

The Subtle Art of Not Giving a F*ck: A Cou...
The Seven Husbands of Evelyn Hugo: A N...
Atlas of the Heart: Mapping Meaningful Co...
A Game of Thrones: A Song of Ice and Fir...
Atomic Habits: An Easy & Proven Way to ...
Girl, Wash Your Face: Stop Believing the L...
The Power of One More: The Ultimate Gui...
The Body Keeps the Score: Brain, Mind, a...
The Power of Now: A Guide to Spiritual En...
The Killing Game (An Alexa Chase Suspen...
The Choice: The Dragon Heart Legacy, Bo...
Sands of Dune: Novellas from the Worlds ...
Friends, Lovers, and the Big Terrible Thin...
What Happened To You?: Conversations o...
Family of Liars: The Prequel to We Were L...
Left To Die (An Adele Sharp Mystery-Book...
How to Not Die Alone: The Surprising Scie...
Killing the Killers: The Secret War Against ...
Feeding the Soul (Because It's My Busine...
Hard to Break: Why Our Brains Make Habi...
The TenX Rule: The Only Difference Betwe...
Rich Dad, Poor Dad. The 20th Anniversary...
The Devil in the White City

In [80]:
clean_titles_1 = []

for title in titles:
    title = title.replace("...","")#removing brings you to product page 
    title = title.replace("[","") #these result in an error when included in search on audiobooks.com
    title = title.replace("]","")
    clean_titles_1.append(title)

In [81]:
len(clean_titles_1)

2000

## Webscrape to get full audiobook details using cleaned titles

In [83]:
#define function to pull the details of interest

def get_product_page_details():
    try:
        rank = counter
    except:
        rank = "not found"
    try:
        new_title = driver.find_element_by_class_name("audiobookTitle").text
    except:
        new_title = title
    try:
        author =  driver.find_element_by_class_name("book-written-by").text
    except:
        author = "not found"
    try:
        price = driver.find_element_by_class_name("book-actions").text 
    except:
        price = "not found"
    try:
        date = driver.find_element_by_class_name("book-published-by").text
    except:
        date = "not found"
    try:
        duration = driver.find_element_by_class_name("book-duration").text
    except:
        duration = "not found"
    try:
        narrator = driver.find_element_by_class_name("book-narrated-by").text
    except:
        narrator = "not found"
    try:
        genre = driver.find_element_by_id("genre_0").text
    except:
        genre = "not found"
    try:
        rating = driver.find_element_by_css_selector("#content > div > div > div.col_one_third.center > div.book-details-ratings > div.total-book-rating.book-rating")
        fin_rating = rating.get_attribute('data-score')
    except:
        fin_rating = "not_found"
    unique_book_dict = {"rank":rank,"title":new_title,"author":author,"price":price,"date":date,"narrator":narrator,"genre":genre,"duration":duration,"rating":fin_rating}
    return unique_book_dict

In [88]:
#Initiate an automated webdriver

chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = \
    chromedriver
driver = webdriver.Chrome(chromedriver)

/var/folders/lh/579wc67x6xgc_8dnfp0724fc0000gn/T/ipykernel_6568/2591033429.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [89]:
#Initiate the webscrape

book_details = []
driver.get('https://www.audiobooks.com/browse/listType/list/page/100')
counter = 0

for title in clean_titles_1:
    time.sleep(1 + 2*random.random())
    counter +=1
    if (counter % 250 == 0):
        time.sleep(320)
    try:
        element = driver.find_element_by_xpath("//*[@id='searchText']")
        element.send_keys(title)
        driver.find_element_by_class_name("icon-search").click()
    except:
        pass
    try:
        is_search_page = driver.find_element_by_class_name("search-results-title").text
        driver.find_element_by_tag_name("h5").click()
        x = get_product_page_details()
    except:
        x = get_product_page_details() 
    try:
        book_details.append(x) 
    except:
        unique_book_dict = {"title":title,"author":"author","price":"price","date":"date",
                            "narrator":"narrator","genre":"genre","rating":"rating","duration":"duration","rank":counter}
        book_details.append(unique_book_dict)
    if (counter % 200 == 0):
        with open('book_details_dict.pkl','wb') as f:
            pickle.dump(book_details,f)


/var/folders/lh/579wc67x6xgc_8dnfp0724fc0000gn/T/ipykernel_6568/2006590434.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = driver.find_element_by_xpath("//*[@id='searchText']")
/var/folders/lh/579wc67x6xgc_8dnfp0724fc0000gn/T/ipykernel_6568/2006590434.py:13: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("icon-search").click()
/var/folders/lh/579wc67x6xgc_8dnfp0724fc0000gn/T/ipykernel_6568/2006590434.py:17: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  is_search_page = driver.find_element_by_class_name("search-results-title").text
/var/folders/lh/579wc67x6xgc_8dnfp0724fc0000gn/T/ipykernel_6568/1309329394.py:7: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME,

In [ ]:
driver.close()